## New Updated code


#### importing necessary libraries

In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import random
import socket
import json
from aes_encryption import encrypt_message, decrypt_message

# Load config values
with open("config.json") as f:
    config = json.load(f)

BANK_IP = config["BANK_SERVER_IP"]
BANK_PORT = config["BANK_SERVER_PORT"]
UPI_IP = config["UPI_SERVER_IP"]
UPI_PORT = config["UPI_SERVER_PORT"]

#### Main Page

In [2]:
def show_main_page():
    for widget in root.winfo_children():
        widget.destroy()
    
    root.title("User Laptop")
    
    frame = tk.Frame(root, padx=20, pady=20)
    frame.pack(expand=True)
    
    label = tk.Label(frame, text="User Laptop", font=("Arial", 18, "bold"))
    label.grid(row=0, column=0, columnspan=2, pady=10)
    
    register_button = ttk.Button(frame, text="User Registration", command=show_registration_page)
    register_button.grid(row=1, column=0, padx=10, pady=10)
    
    transaction_button = ttk.Button(frame, text="Transaction", command=show_transaction_page)
    transaction_button.grid(row=1, column=1, padx=10, pady=10)

#### Registration Page

In [3]:
def show_registration_page():
    ifsc_codes = {
    "HDFC": ["HDFC001", "HDFC002", "HDFC003"],
    "ICICI": ["ICICI001", "ICICI002", "ICICI003"],
    "SBI": ["SBI001", "SBI002", "SBI003"]
    }
    
    def update_ifsc(*args):
        selected_bank = bank_var.get()
        ifsc_dropdown["values"] = ifsc_codes.get(selected_bank, [])
    
    def register_user():
        name = name_entry.get()
        password = password_entry.get()
        balance = balance_entry.get()
        ifsc = ifsc_dropdown.get()
        pin = pin_entry.get()
        
        # Check for empty fields
        if not (name and password and balance and ifsc and pin):
            messagebox.showerror("Error", "All fields are required")
            return

        # Validate PIN
        if not pin.isdigit() or len(pin) != 4:
            messagebox.showerror("Error", "PIN must be a 4-digit number")
            return
        
        # Validate balance
        try:
            balance = float(balance)
            if balance < 0:
                raise ValueError
        except ValueError:
            messagebox.showerror("Error", "Balance must be a non-negative numeric value")
            return
        
        # Create user dictionary
        user_details = {
            "type": "user",
            "name": name,
            "password": password,
            "balance": balance,
            "ifsc": ifsc,
            "pin": pin
        }

        try:
            # Connect to Bank Server
            client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            client_socket.connect((BANK_IP, BANK_PORT))  # Connect to bank server
            client_socket.send(json.dumps(user_details).encode("utf-8"))  # Send JSON data
            
            # Receive response (MID)
            response = json.loads(client_socket.recv(1024).decode())
            messagebox.showinfo("Registration Successful", f"User ID: {response['UID']}")
            name_entry.delete(0, tk.END)
            password_entry.delete(0, tk.END)
            balance_entry.delete(0, tk.END)
            pin_entry.delete(0, tk.END)
            bank_var.set('')
            ifsc_dropdown.set('')
            client_socket.close()

        except Exception as e:
            messagebox.showerror("Error", f"Failed to register: {e}")
        
    
    for widget in root.winfo_children():
        widget.destroy()
    
    frame = tk.Frame(root, padx=20, pady=20)
    frame.pack(expand=True)
    
    tk.Label(frame, text="User Registration", font=("Arial", 16, "bold")).grid(row=0, columnspan=2, pady=10)
    
    tk.Label(frame, text="User Name:").grid(row=1, column=0, pady=5, sticky="e")
    name_entry = ttk.Entry(frame)
    name_entry.grid(row=1, column=1, pady=5)
    
    tk.Label(frame, text="Password:").grid(row=2, column=0, pady=5, sticky="e")
    password_entry = ttk.Entry(frame, show="*")
    password_entry.grid(row=2, column=1, pady=5)
    
    tk.Label(frame, text="Initial Balance:").grid(row=3, column=0, pady=5, sticky="e")
    balance_entry = ttk.Entry(frame)
    balance_entry.grid(row=3, column=1, pady=5)
    
    tk.Label(frame, text="Bank:").grid(row=4, column=0, pady=5, sticky="e")
    bank_var = tk.StringVar()
    bank_dropdown = ttk.Combobox(frame, textvariable=bank_var, values=list(ifsc_codes.keys()))
    bank_dropdown.grid(row=4, column=1, pady=5)
    bank_dropdown.bind("<<ComboboxSelected>>", update_ifsc)
    
    tk.Label(frame, text="IFSC Code:").grid(row=5, column=0, pady=5, sticky="e")
    ifsc_dropdown = ttk.Combobox(frame)
    ifsc_dropdown.grid(row=5, column=1, pady=5)
    
    tk.Label(frame, text="PIN:").grid(row=6, column=0, pady=5, sticky="e")
    pin_entry = ttk.Entry(frame, show="*")
    pin_entry.grid(row=6, column=1, pady=5)
    
    uid_label = tk.Label(frame, text="")
    uid_label.grid(row=7, columnspan=2, pady=5)
    
    ttk.Button(frame, text="Register", command=register_user).grid(row=8, columnspan=2, pady=10)
    ttk.Button(frame, text="Generate MMID", command=show_mmid_page).grid(row=9, columnspan=2, pady=5)
    ttk.Button(frame, text="Back", command=show_main_page).grid(row=10, columnspan=2, pady=5)

#### generate mmid page

In [4]:
def show_mmid_page():
    def check_user_id(uid):
        try:
            client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            client_socket.connect((BANK_IP, BANK_PORT))  # Connect to Bank Server

            details = {
                "type": "check_user",
                "user_id": uid
            }

            client_socket.send(json.dumps(details).encode("utf-8"))

            response = client_socket.recv(1024).decode("utf-8")
            client_socket.close()

            return response

        except Exception as e:
            print("Error checking user ID:", e)
            return None
        
    def generate_mmid():
        user_id = uid_entry.get()
        mobile_number = mobile_entry.get()
        
        check_uid = check_user_id(user_id)
        
        if check_uid != "True":
            messagebox.showerror("Error", "User not registered")
            return
        
        if not mobile_number:
            messagebox.showerror("Error", "Enter Mobile Number")
            return
        
        # Validate mobile number
        if not mobile_number.isdigit() or len(mobile_number) != 10:
            messagebox.showerror("Error", "Enter a valid 10-digit mobile number")
            return

        
        try:
            # Connect to bank server
            client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            client_socket.connect((BANK_IP, BANK_PORT))
            
            other_details = {
                "type": "MMID_Generation",
                "user_id": user_id,
                "mobile_number": mobile_number
            }

            # Send merchant details to bank
            client_socket.send(json.dumps(other_details).encode())

            # Receive response (MID)
            response = json.loads(client_socket.recv(1024).decode())
            messagebox.showinfo("Registration Successful", f"MMID: {response['MMID']}")

            client_socket.close()
        except Exception as e:
            messagebox.showerror("Connection Error", f"Could not connect to bank: {e}")
    
    for widget in root.winfo_children():
        widget.destroy()
    
    frame = tk.Frame(root, padx=20, pady=20)
    frame.pack(expand=True)
    
    tk.Label(frame, text="Generate MMID", font=("Arial", 16, "bold")).grid(row=0, columnspan=2, pady=10)
    
    tk.Label(frame, text="User ID:").grid(row=1, column=0, pady=5, sticky="e")
    uid_entry = ttk.Entry(frame)
    uid_entry.grid(row=1, column=1, pady=5)
    
    tk.Label(frame, text="Mobile Number:").grid(row=2, column=0, pady=5, sticky="e")
    mobile_entry = ttk.Entry(frame)
    mobile_entry.grid(row=2, column=1, pady=5)
    
    ttk.Button(frame, text="Generate MMID", command=generate_mmid).grid(row=3, columnspan=2, pady=10)
    ttk.Button(frame, text="Back", command=show_registration_page).grid(row=4, columnspan=2, pady=5)


#### Transaction Page

In [5]:
def show_transaction_page():
    def process_transaction():
        mmid = mmid_entry.get()
        pin = pin_entry.get()
        amount = amount_entry.get()
        vmid = vmid_entry.get()
        
        encrypt_amount = encrypt_message(amount)
        
        if not (mmid and pin and amount and vmid):
            messagebox.showerror("Error", "All fields are required")
            return

        try:
            amount = float(amount)
        except ValueError:
            messagebox.showerror("Error", "Invalid amount")
            return
        
        encrypt_mmid = encrypt_message(mmid)
        encrypt_pin = encrypt_message(pin)
        
        
        try:
            client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            client.connect((UPI_IP,UPI_PORT))  # Connect to UPI server

            transaction_data = {
                "mmid": encrypt_mmid,
                "pin": encrypt_pin,
                "amount": encrypt_amount,
                "merchant_id": vmid
            }

            client.send(json.dumps(transaction_data).encode())
            messagebox.showinfo("Info", "Transaction sent to UPI machine. Please wait for confirmation.")
            client.close()

        except Exception as e:
            messagebox.showerror("Error", f"Transaction has failed: {e}")

    
    for widget in root.winfo_children():
        widget.destroy()
    
    frame = tk.Frame(root, padx=20, pady=20)
    frame.pack(expand=True)
    
    tk.Label(frame, text="Transaction", font=("Arial", 16, "bold")).grid(row=0, columnspan=2, pady=10)
    tk.Label(frame, text="MMID:").grid(row=1, column=0, pady=5)
    mmid_entry = ttk.Entry(frame)
    mmid_entry.grid(row=1, column=1, pady=5)
    tk.Label(frame, text="Merchant Id:").grid(row=2, column=0, pady=5)
    vmid_entry = ttk.Entry(frame)
    vmid_entry.grid(row=2, column=1, pady=5)
    tk.Label(frame, text="PIN:").grid(row=3, column=0, pady=5)
    pin_entry = ttk.Entry(frame, show="*")
    pin_entry.grid(row=3, column=1, pady=5)
    tk.Label(frame, text="Amount:").grid(row=4, column=0, pady=5)
    amount_entry = ttk.Entry(frame)
    amount_entry.grid(row=4, column=1, pady=5)
    ttk.Button(frame, text="Send & Pay via UPI", command=process_transaction).grid(row=5, columnspan=2, pady=10)
    ttk.Button(frame, text="Back", command=show_main_page).grid(row=6, columnspan=2, pady=5)


#### Run

In [7]:
root = tk.Tk()
root.geometry("500x600")
show_main_page()
root.mainloop()